In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd

from scipy.spatial import KDTree, cKDTree
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import time
import joblib
from tqdm import tqdm
import trackpy as tp
from numba import njit, prange

from yupi import Trajectory
import yupi.graphics as yg
import yupi.stats as ys

from utility import get_imsd, get_imsd_windowed, get_emsd, get_emsd_windowed, fit_hist, MB_2D,\
                    normal_distr, lorentzian_distr, get_trajs, speed_windowed, theta_windowed, \
                    get_smooth_trajs, get_velocities

In [7]:
traj_verb = "stardist"

if traj_verb == "trackpy": 
    rawTrajs = pd.read_parquet("../tracking/results/tracking_data/trackpy_pre_merge.parquet")
    nDrops = int(len(rawTrajs.loc[rawTrajs.frame==0]))
    red_particle_idx = 17
    # Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
    smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)

    res_path = "results"
    analysis_data_path = "analysis_data"

elif traj_verb == "hough":
    rawTrajs = pd.read_parquet("../tracking/results/tracking_data//hough/linked_pre_merge.parquet")#pd.read_parquet("../tracking/results/tracking_data/tracking_hough_trackpy_linking.parquet")
    nDrops = int(len(rawTrajs.loc[rawTrajs.frame==0]))
    red_particle_idx = 17
    # Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
    smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)
    
    res_path = "hough_results"
    analysis_data_path = "hough_analysis_data"

elif traj_verb == "stardist":
    rawTrajs = pd.read_parquet("../tracking/stardist_res/noblur/df_linked.parquet")
    nDrops = int(len(rawTrajs.loc[rawTrajs.frame==0]))
    # Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
    smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)
    red_particle_idx = 10

    res_path = "stardist_results" 
    analysis_data_path = "stardist_analysis_data"
else:
    raise ValueError("traj_verb must be either 'trackpy' or 'hough'")


colors = rawTrajs.loc[rawTrajs.frame == 0, 'color'].values
nFrames = int(max(rawTrajs.frame) + 1)
fps = 10
print(f"Number of Droplets: {nDrops}")
print(f"Number of Frames: {nFrames} at {fps} fps --> {nFrames/10:.2f} s")

# WINDOWED ANALYSIS PARAMETERS
window = 3200 # 320 s
stride = 100 # 10 s
print("Windowed analysis args:")

startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"window of {window/10} s, stride of {stride/10} s --> {nSteps} steps")

# step 10 with a 10 fps video --> 1 s
units = "px/s"
default_kwargs_blue = {"color": "#00FFFF", "ec": (0, 0, 0, 0.6), "density": True}
default_kwargs_red = {"color": "#EE4B2B", "ec": (0, 0, 0, 0.6), "density": True}

Number of Droplets: 50
Number of Frames: 30000 at 10 fps --> 3000.00 s
Windowed analysis args:
window of 320.0 s, stride of 10.0 s --> 268 steps


# MSD

In [9]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True

print(f"MSD Analysis: trajs_verb = {traj_verb}, show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/msd.ipynb
#%run ./analysis_modules/msd.py

MSD Analysis: trajs_verb = stardist, show_verb = False, run_windowed_analysis = True, animated_plot_verb = True
Global IMSD
Global EMSD
Raw trajs - Blue Particles: a = 1.178 ± 0.002, K = 61.491 ± 0.431
Raw trajs - Red Particle: a = 0.763 ± 0.001, K = 1475.813 ± 8.621
Smooth trajs - Blue Particles: a = 1.18 ± 0.002, K = 61.032 ± 0.435
Smooth trajs - Red Particle: a = 0.765 ± 0.001, K = 1455.679 ± 8.736
Windowed IMSD


100%|██████████| 268/268 [04:35<00:00,  1.03s/it]
12:21:53 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
12:21:53 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/mean_squared_displacement/windowed_analysis/IMSD_wind_raw.mp4
12:22:28 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
12:22:28 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/mean_squared_displacement/windowed_analysis/IMSD_wind_smooth.mp4


Windowed EMSD


12:23:11 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
12:23:11 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/mean_squared_displacement/windowed_analysis/EMSD_wind_raw.mp4
12:23:37 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
12:23:37 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/mean_squared_displacement/windowed_analysis/EMSD_wind_smooth.mp4


# VELOCITY DISTRIBUTION

$$ \huge p(v) = \frac{v}{\sigma^2} e^{-\frac{v^2}{2\sigma^2}} $$

In [10]:
show_verb = False
plot_verb = True
animated_plot_verb = True
save_verb = True

v_step = 10

print(f"Speed Analysis: trajs_verb = {traj_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_distr.ipynb

Speed Analysis: trajs_verb = stardist, show_verb = False, animated_plot_verb = True

 Global speed distribution analysis
Raw Trajectories
Blue Particles σ: 2.840536643626226 ± 0.019486900067007207
Red Particle σ: 8.523510198137256 ± 0.17484495828328145
Smooth Trajectories
Blue Particles σ: 2.695091926784261 ± 0.018440786910659774
Red Particle σ: 8.582079069410534 ± 0.17660785993843936

 Windowed speed distribution Analysis


100%|██████████| 268/268 [00:58<00:00,  4.62it/s]
15:18:09 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
15:18:09 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/speed_distribution/speed_raw_wind.mp4
15:20:16 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
15:20:16 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/speed_distribution/speed_smooth_wind.mp4


# TURNING ANGLES DISTRIBUTIONS

In [11]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True

# choose the distribution to fit
distribution_str = "lorentzian"

if distribution_str == "gaussian":
    distribution = normal_distr
elif distribution_str == "lorentzian":
    distribution = lorentzian_distr
    distribution_str = "lorentzian"
else:
    raise ValueError("distribution_str must be either 'gaussian' or 'lorentzian'")

print(f"Turning Angles Analysis: trajs_verb = {traj_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/turning_angles.ipynb

Turning Angles Analysis: trajs_verb = stardist, show_verb = False, animated_plot_verb = True

 Global turning angles analysis
Raw Trajectories
Blue Particles σ: 0.2603550046364755 ± 0.0031217060943062873, μ: 0.000860941389088082 ± 0.003120981977861532
Red Particle σ: 0.17493170562112526 ± 0.0023488120487898385, μ: -0.005456977614727908 ± 0.0023484620771724737
Smooth Trajectories
Blue Particles σ: 0.07154922678931545 ± 0.0005369027908970597, μ: -7.924505823246546e-05 ± 0.0004420583223647755
Red Particle σ: 0.053794393098102744 ± 0.0007349125671607246, μ: 0.0017895247510092355 ± 0.0004343643959283088

 Windowed turning angles analysis


100%|██████████| 268/268 [00:37<00:00,  7.09it/s]
15:25:03 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
15:25:03 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/turning_angles/lorentzian/turn_ang_wind_raw.mp4
15:25:30 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
15:25:30 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/turning_angles/lorentzian/turn_ang_wind_smooth.mp4


# VELOCITY AUTOCORRELATION

In [13]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True
overwrite = False

print(f"Velocity Autocorrelation Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_autocorrelation.ipynb

Velocity Autocorrelation Analysis: show_verb = False, run_windowed_analysis = True, animated_plot_verb = True
Global Velocity Autocovariance Function
Windowed analysis


100%|██████████| 268/268 [31:15<00:00,  7.00s/it]
16:32:25 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
16:32:25 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/velocity_autocovariance/vacf_wind_raw.mp4
16:32:46 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
16:32:46 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/velocity_autocovariance/vacf_wind_smooth.mp4


# RADIAL DISTRIBUTON FUNCTION

In [17]:
run_analysis_verb = True
show_verb = False
animated_plot_verb = True
save_verb = True

print(f"Radial Distribution Function Analysis: {traj_verb}, run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/radial_distribution_function.ipynb

Radial Distribution Function Analysis: stardist, run_analysis_verb = True, show_verb = False, animated_plot_verb = True
RDF - Raw Trajectories
raw


100%|██████████| 30000/30000 [1:13:38<00:00,  6.79it/s]
19:22:14 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
19:22:14 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1000x400 -pix_fmt rgba -r 60 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/radial_distribution_function/rdf_raw.mp4


RDF from center - Raw Trajectories


100%|██████████| 30000/30000 [02:51<00:00, 174.57it/s]


RDF from center - Smooth Trajectories


100%|██████████| 30000/30000 [02:43<00:00, 183.09it/s]
19:29:17 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
19:29:17 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 60 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/radial_distribution_function/rdf_from_center_raw.mp4
19:30:34 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
19:30:34 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 60 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/radial_distribution_function/rdf_from_center_smooth.mp4


# 3 BODY STRUCTURE METRIC

In [ ]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

%run ./analysis_modules/3body_analysis.ipynb